In [ ]:
import talib
import pandas as pd
from sqlalchemy import create_engine
import ast  # To safely evaluate strings as tuples

# Load raw data from the database
engine = create_engine('sqlite:///stocks.db')
raw_data = pd.read_sql('raw_data', engine)

# Print the original column names
print("Original columns:", raw_data.columns)

# Debug: Print the first few rows of the DataFrame
print("First few rows of raw_data:")
print(raw_data.head())

# Step 1: Convert column names from strings to tuples
# Example: "('Date', '')" becomes ('Date', '')
raw_data.columns = [ast.literal_eval(col) if isinstance(col, str) and col.startswith('(') else col for col in raw_data.columns]

# Print the cleaned column names
print("Cleaned columns (as tuples):", raw_data.columns)

# Step 2: Create a new DataFrame with clean column names
clean_columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'Ticker']
new_data = pd.DataFrame(columns=clean_columns)

# Step 3: Copy data from the old DataFrame into the new one
# Iterate through each row in the old DataFrame
for index, row in raw_data.iterrows():
    # Extract values from the old DataFrame
    date = row[('Date', '')]
    close = row[('Close', 'AAPL')]  # Replace 'AAPL' with the appropriate ticker
    high = row[('High', 'AAPL')]
    low = row[('Low', 'AAPL')]
    open_ = row[('Open', 'AAPL')]
    volume = row[('Volume', 'AAPL')]
    ticker = row[('Ticker', '')]
    
    # Create a new row as a DataFrame
    new_row = pd.DataFrame({
        'Date': [date],
        'Close': [close],
        'High': [high],
        'Low': [low],
        'Open': [open_],
        'Volume': [volume],
        'Ticker': [ticker]
    })
    
    # Append the new row to the new DataFrame using pd.concat
    # Exclude empty or all-NA entries to avoid FutureWarning
    new_data = pd.concat([new_data, new_row], ignore_index=True, join='inner')

# Print the new DataFrame
print("New DataFrame with clean columns:")
print(new_data.head())

# Step 4: Proceed with feature engineering
def calculate_indicators(df):
    # Use the appropriate column names for each ticker
    ticker = df['Ticker'].iloc[0]
    close_col = 'Close'  # No need to append ticker since we have a clean DataFrame
    
    df['SMA'] = talib.SMA(df[close_col], timeperiod=30)
    df['RSI'] = talib.RSI(df[close_col], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(df[close_col])
    return df

# Group by 'Ticker' and apply the function
# Exclude grouping columns to avoid DeprecationWarning
processed_data = new_data.groupby('Ticker', group_keys=False).apply(calculate_indicators)

# Save processed data to the database
processed_data.to_sql('processed_data', engine, if_exists='replace', index=False)

print("Processed data saved to the database successfully!")